# Loading and filtering 

In [38]:
import h5py
import numpy as np

In [109]:
%ls -lh ../../

total 2289784
drwxr-xr-x   4 thomasvuillaume  staff   128B Sep 14 10:46 Biblio/
drwxr-xr-x  12 thomasvuillaume  staff   384B Dec  1 17:10 GammaLearn/
-rw-r--r--   1 thomasvuillaume  staff   1.1G Dec  1 17:09 LaPalma_gamma_diffuse_20deg_0deg_prod3b_training_0260.hdf5
-rw-r--r--@  1 thomasvuillaume  staff    11K Jul 22 09:13 OBELICS - support.xlsx
drwxr-xr-x  10 thomasvuillaume  staff   320B Sep 27 14:49 Project_papers/
drwxr-xr-x   7 thomasvuillaume  staff   224B Sep 12 11:00 gammalearn-articles/
drwxr-xr-x   8 thomasvuillaume  staff   256B Sep 20 15:09 gammalearn-meetings/
drwxr-xr-x   6 thomasvuillaume  staff   192B Nov 15 14:42 icon/


In [111]:
testfile = '../../LaPalma_gamma_diffuse_20deg_0deg_prod3b_training_0260.hdf5'

In [112]:
ls -lh testfile

ls: testfile: No such file or directory


In [3]:
f = h5py.File(testfile, 'r')

In [20]:
for n in f:
    print(n)
    for n2 in f[n]:
        print("\t", n2)

Cameras
	 FLASH
	 LSTCAM
	 NECTAR
	 SCT
eventSimu
	 eventId
	 runId
	 showerId
	 xCore
	 yCore
pcalibrun_files
psimu_files
showerSimu
	 altitude
	 azimuth
	 cmax
	 depthStart
	 emax
	 energy
	 heightFirstInteraction
	 hmax
	 runId
	 showerId
	 xmax
telescopeInfos
	 telescopeFocal
	 telescopeId
	 telescopePosition


In [21]:
for n in f['Cameras/LSTCAM']:
    print(n)

eventId
images
injTable
pixelsPosition
runId
telescopeAltitude
telescopeAzimuth
telescopeId


In [35]:
energy = f['showerSimu/energy']
runId  = f['showerSimu/runId']
showerId = f['showerSimu/showerId']
print(len(energy), len(runId), len(showerId))

2843 2843 2843


In [84]:
EnergyMask = (f['showerSimu/energy'].value > 0.95) & (f['showerSimu/energy'].value < 1)

In [107]:
len(f['Cameras/LSTCAM/images'])

6821

In [97]:
def filter_energy(file, emin=-np.inf, emax=np.inf):
    EnergyMask = (file['showerSimu/energy'].value > emin) & (file['showerSimu/energy'].value < emax)
    EventMask = np.in1d(file['eventSimu/showerId'].value, showerId[EnergyMask])
    eventId = file['eventSimu/eventId'][EventMask]
    ImageMask = np.in1d(file['Cameras/LSTCAM/eventId'], eventId)
    
    return file['Cameras/LSTCAM/images'].value[ImageMask]

In [105]:
%timeit Images = filter_energy(f, emin=0.9, emax=1)

50.9 ms ± 1.05 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [106]:
len(Images)

3580